# Install Packages and Setup Variables


In [ ]:
!pip install -q llama-index==0.10.65 llama-index-finetuning llama-index-embeddings-adapter openai==1.37.0 tiktoken==0.7.0 chromadb==0.5.5 llama-index-vector-stores-chroma==0.1.10 cohere==5.6.2 llama-index-llms-gemini==0.1.11 html2text llama-index-llms-openai llama-index-embeddings-huggingface llama-index-embeddings-openai llama-index-llms-azure-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

# Download the Dataset


In [ ]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl:   0%|          | 0.00/6.96M [00:00<?, ?B/s]

In [ ]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

## LlamaIndex Document

In [ ]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### Splitting Dataset


In [ ]:
import random

random.shuffle(doc)
split_index = int(len(doc) * 0.9)

# TRAIN_DOCs and VALIDATION_DOCs
TRAIN_DOCs = doc[:split_index]
VALIDATION_DOCs = doc[split_index:]

# Chunking


In [ ]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import Document

# Now use the parser
parser = SimpleNodeParser.from_defaults(chunk_size=768, chunk_overlap=64)
TRAIN_NODEs = parser.get_nodes_from_documents(TRAIN_DOCs)
VALIDATION_NODEs = parser.get_nodes_from_documents(VALIDATION_DOCs)

print(len(TRAIN_NODEs), len(VALIDATION_NODEs))

2771 340


In [ ]:
# Use a subset of the dataset if testing.

# Test with a few sample, processing dataset fully can be costly depanding on the size.
# NOTE: Checkpoints are provided in the lesson, so no need to run the code on full dataset.

testing =False

if testing:
    TRAIN_NODEs = TRAIN_NODEs[0:10]
    VALIDATION_NODEs = VALIDATION_NODEs[0:5]

# Generate Question


We use a Large Language Model (LLM) to produce questions for each chunk of the dataset. Then we can use these data to train the model to develop embeddings that more accurately represent the types of questions users may ask.


In [ ]:
# Use this block of code if you don't want to generate the questions for the dataset. (Avoid API call charges!)
# Uncomment the following code, and keep in mind to comment all the contents in the next coding block.


# from llama_index.finetuning import generate_qa_embedding_pairs
# from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-4o-mini", temperature=1, max_tokens=512)

# # Generate questions for each chunk.

# TRAIN_DATASET = generate_qa_embedding_pairs(TRAIN_NODEs, llm=llm, output_path="./train_dataset.json")

# VALIDATION_DATASET = generate_qa_embedding_pairs(VALIDATION_NODEs, llm=llm, output_path="./val_dataset.json")

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="jaiganesan/Embedding-model-fine-tuning-dataset", repo_type="dataset",local_dir="/content/")


from llama_index.finetuning import EmbeddingQAFinetuneDataset

# Load the pre-generated questions json files.
TRAIN_DATASET = EmbeddingQAFinetuneDataset.from_json("./train_dataset.json")
VALIDATION_DATASET = EmbeddingQAFinetuneDataset.from_json("./val_dataset.json")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

val_dataset.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

train_dataset.json:   0%|          | 0.00/8.23M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

# Load an Embedding Model


In [ ]:
from llama_index.core.embeddings import resolve_embed_model

# Load an existing embedding model with a linear layer adopter on top.
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
import torch

# Finetune the adapter
finetune_engine = EmbeddingAdapterFinetuneEngine(
    TRAIN_DATASET,
    base_embed_model,
    model_output_path="model_output_test",
    epochs=2,
    verbose=True,
    bias=True,
)

In [ ]:
# Initiate the Finetuning process
finetune_engine.finetune()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.5561193227767944
> [Epoch 0] Current loss: 1.5454338788986206
> [Epoch 0] Current loss: 1.3019956350326538
> [Epoch 0] Current loss: 1.210197925567627
> [Epoch 0] Current loss: 1.0392591953277588
> [Epoch 0] Current loss: 1.0869503021240234
> [Epoch 0] Current loss: 1.6084495782852173
> [Epoch 0] Current loss: 1.3948441743850708
> [Epoch 0] Current loss: 1.624601125717163
> [Epoch 0] Current loss: 1.833229660987854
> [Epoch 0] Current loss: 2.3522706031799316
> [Epoch 0] Current loss: 2.3650951385498047
> [Epoch 0] Current loss: 2.2668750286102295
> [Epoch 0] Current loss: 1.8664640188217163
> [Epoch 0] Current loss: 2.3703010082244873
> [Epoch 0] Current loss: 2.3469536304473877
> [Epoch 0] Current loss: 2.4227194786071777
> [Epoch 0] Current loss: 2.3735411167144775
> [Epoch 0] Current loss: 2.2941596508026123
> [Epoch 0] Current loss: 2.360380172729492
> [Epoch 0] Current loss: 2.3185575008392334
> [Epoch 0] Current loss: 2.3356430530548096
> [Epoch 0] Cu

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.5034734010696411
> [Epoch 1] Current loss: 1.4798120260238647
> [Epoch 1] Current loss: 1.2930068969726562
> [Epoch 1] Current loss: 1.163027048110962
> [Epoch 1] Current loss: 1.0291184186935425
> [Epoch 1] Current loss: 1.0037190914154053
> [Epoch 1] Current loss: 1.52053964138031
> [Epoch 1] Current loss: 1.3807648420333862
> [Epoch 1] Current loss: 1.6184279918670654
> [Epoch 1] Current loss: 1.8021923303604126
> [Epoch 1] Current loss: 2.352696418762207
> [Epoch 1] Current loss: 2.3439199924468994
> [Epoch 1] Current loss: 2.264328718185425
> [Epoch 1] Current loss: 1.8568464517593384
> [Epoch 1] Current loss: 2.3480913639068604
> [Epoch 1] Current loss: 2.3382060527801514
> [Epoch 1] Current loss: 2.3973586559295654
> [Epoch 1] Current loss: 2.3641159534454346
> [Epoch 1] Current loss: 2.272991418838501
> [Epoch 1] Current loss: 2.3372738361358643
> [Epoch 1] Current loss: 2.310943603515625
> [Epoch 1] Current loss: 2.3237786293029785
> [Epoch 1] Curre

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

# Or, import model from the directory whenever required.
# from llama_index.core.embeddings import LinearAdapterEmbeddingModel
# embed_model = LinearAdapterEmbeddingModel(base_embed_model, "model_output_test")

/usr/local/lib/python3.10/dist-packages/llama_index/embeddings/adapter/utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


In [ ]:
embed_model

AdapterEmbeddingModel(model_name='Adapter for BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ebe7951f220>, num_workers=None)

## Fine tuning OpenAI Embedding Model using Adapter method

In [ ]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.embeddings.openai import OpenAIEmbedding

openai_finetune_engine = EmbeddingAdapterFinetuneEngine(
    TRAIN_DATASET,
    OpenAIEmbedding(model="text-embedding-3-small"),
    model_output_path="model_output_test_openai",
    bias=True,
    epochs=2,
    verbose=True,
)

In [ ]:
openai_finetune_engine.finetune()

openai_embed_model = openai_finetune_engine.get_finetuned_model()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.1989654302597046
> [Epoch 0] Current loss: 1.2277913093566895
> [Epoch 0] Current loss: 0.9640659093856812
> [Epoch 0] Current loss: 0.932266354560852
> [Epoch 0] Current loss: 1.119067907333374
> [Epoch 0] Current loss: 0.9123556017875671
> [Epoch 0] Current loss: 0.8483080863952637
> [Epoch 0] Current loss: 1.1643975973129272
> [Epoch 0] Current loss: 1.1628005504608154
> [Epoch 0] Current loss: 1.7469831705093384
> [Epoch 0] Current loss: 2.2921977043151855
> [Epoch 0] Current loss: 2.3667550086975098
> [Epoch 0] Current loss: 2.324866771697998
> [Epoch 0] Current loss: 1.804431676864624
> [Epoch 0] Current loss: 2.3605501651763916
> [Epoch 0] Current loss: 2.314915180206299
> [Epoch 0] Current loss: 2.3383851051330566
> [Epoch 0] Current loss: 2.272596597671509
> [Epoch 0] Current loss: 1.4600045680999756
> [Epoch 0] Current loss: 2.30053973197937
> [Epoch 0] Current loss: 2.281567096710205
> [Epoch 0] Current loss: 2.3898096084594727
> [Epoch 0] Current

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.1400434970855713
> [Epoch 1] Current loss: 1.1969709396362305
> [Epoch 1] Current loss: 0.9479926228523254
> [Epoch 1] Current loss: 0.9089117050170898
> [Epoch 1] Current loss: 1.1214836835861206
> [Epoch 1] Current loss: 0.9035440683364868
> [Epoch 1] Current loss: 0.8298786878585815
> [Epoch 1] Current loss: 1.179967999458313
> [Epoch 1] Current loss: 1.1542689800262451
> [Epoch 1] Current loss: 1.6849565505981445
> [Epoch 1] Current loss: 2.2848310470581055
> [Epoch 1] Current loss: 2.3603434562683105
> [Epoch 1] Current loss: 2.316659450531006
> [Epoch 1] Current loss: 1.7936054468154907
> [Epoch 1] Current loss: 2.3495335578918457
> [Epoch 1] Current loss: 2.303473949432373
> [Epoch 1] Current loss: 2.3347811698913574
> [Epoch 1] Current loss: 2.2543702125549316
> [Epoch 1] Current loss: 1.439344048500061
> [Epoch 1] Current loss: 2.288567066192627
> [Epoch 1] Current loss: 2.2726378440856934
> [Epoch 1] Current loss: 2.381418466567993
> [Epoch 1] Curr

/usr/local/lib/python3.10/dist-packages/llama_index/embeddings/adapter/utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


In [ ]:
openai_embed_model

AdapterEmbeddingModel(model_name='Adapter for text-embedding-3-small', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c82b14e01f0>, num_workers=None)

# Evaluate


## Define the Evaluation Functions


In [ ]:
from llama_index.core import ServiceContext, VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm import tqdm
import pandas as pd

def evaluate(dataset, embed_model, top_k=5, verbose=False):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    # Chunking the documents and generating embeddings
    service_context = ServiceContext.from_defaults(embed_model=embed_model)
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)

    # Define a retriever to answer the questions
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []

    # Look into each response sources to see if the chunk that contains the answer is retrieved.
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]

        try:
            rank = retrieved_ids.index(expected_id) + 1
            reciprocal_rank = 1 / rank
        except ValueError:
            rank = None
            reciprocal_rank = 0

        is_hit = expected_id in retrieved_ids

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
            "rank": rank,
            "reciprocal_rank": reciprocal_rank
        }
        eval_results.append(eval_result)

    return eval_results

## OpenAI Embedding Model Evaluation


In [ ]:
# Load the OpenAI Ada model and evaluate it.
openai_text_embedding_small = OpenAIEmbedding(model="text-embedding-3-small")
openai_embedding_val_results = evaluate(VALIDATION_DATASET, openai_text_embedding_small)

<ipython-input-46-4d6c34ecbb4a>:11: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [02:07<00:00,  3.90it/s]


In [ ]:
openai_embedding_val_results = [
    result for result in openai_embedding_val_results if isinstance(result, dict)
]

df_openai = pd.DataFrame(openai_embedding_val_results)

hit_rate_openai = df_openai["is_hit"].mean()
mrr_openai = df_openai["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai}")
print(f"MRR: {mrr_openai}")

Hit rate: 0.9354838709677419
MRR: 0.788877688172043


### OpenAI Embedding Model with Fine Tuned Adapter Model Evaluation

In [ ]:
from llama_index.legacy.embeddings.adapter import LinearAdapterEmbeddingModel

openai_text_embedding_small = OpenAIEmbedding(model="text-embedding-3-small")
openai_embed_model = LinearAdapterEmbeddingModel(openai_text_embedding_small, "model_output_test_openai")

val_results_ft_openai = evaluate(VALIDATION_DATASET, openai_embed_model)

/usr/local/lib/python3.10/dist-packages/llama_index/legacy/embeddings/adapter_utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(
<ipython-input-12-c34e84

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [03:15<00:00,  2.54it/s]


In [ ]:
val_results_ft_openai = [
    result for result in val_results_ft_openai if isinstance(result, dict)
]

df_openai_ft = pd.DataFrame(val_results_ft_openai)

hit_rate_openai_ft = df_openai_ft["is_hit"].mean()
mrr_openai_ft = df_openai_ft["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai_ft}")
print(f"MRR: {mrr_openai_ft}")

Hit rate: 0.9637096774193549
MRR: 0.8210685483870968


## Open Source BAAI Model Evaluation


In [ ]:
# Load the Base model without fine-tuning
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
bge_val_results = evaluate(VALIDATION_DATASET, base_embed_model)

<ipython-input-18-c34e84fce376>:12: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [00:15<00:00, 32.36it/s]


In [ ]:
bge_val_results = [
    result for result in bge_val_results if isinstance(result, dict)
]

df_bge = pd.DataFrame(bge_val_results)

hit_rate_bge = df_bge["is_hit"].mean()
mrr_bge = df_bge["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_bge}")
print(f"MRR: {mrr_bge}")

Hit rate: 0.8387096774193549
MRR: 0.7207325268817205


## FineTuned BAAI Adapter Embedding Model Evaluation


In [ ]:
from llama_index.legacy.embeddings.adapter import LinearAdapterEmbeddingModel

# Load the Fine-tuned model.
embed_model = LinearAdapterEmbeddingModel(base_embed_model, "model_output_test")

val_results_finetuned = evaluate(VALIDATION_DATASET, embed_model)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/llama_index/legacy/embeddings/adapter_utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arb

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

100%|██████████| 496/496 [00:10<00:00, 47.26it/s]


In [ ]:
val_results_finetuned = [
    result for result in val_results_finetuned if isinstance(result, dict)
]

df_finetuned = pd.DataFrame(val_results_finetuned)

hit_rate_finetuned = df_finetuned["is_hit"].mean()
mrr_finetuned = df_finetuned["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_finetuned}")
print(f"MRR: {mrr_finetuned}")

Hit rate: 0.8629032258064516
MRR: 0.7499663978494624
